# Tensorflow 入门

参考：

https://mp.weixin.qq.com/s?__biz=MzI2OTc5NTUwNg==&mid=2247484530&idx=2&sn=158434d20b8e8796a3dbcece6e603d4c&chksm=eadbadafddac24b91e9cb249340f120721181d5a1b952570c079ed09059ce0291ef7daef9fcd&scene=21#wechat_redirect

**基本概念：**

1、 图（Graph）：用来表示计算任务，也就我们要做的一些操作。

2 、会话（Session）：建立会话，此时会生成一张空图；在会话中添加节点和边，形成一张图，一个会话可以有多个图，通过执行这些图得到结果。如果把每个图看做一个车床，那会话就是一个车间，里面有若干个车床，用来把数据生产成结果。

3 、Tensor：用来表示数据，是我们的原料。

4、 变量（Variable）：用来记录一些数据和状态，是我们的容器。

5、 feed和fetch：可以为任意的操作(arbitrary operation) 赋值或者从其中获取数据。相当于一些铲子，可以操作数据。

形象的比喻是：把会话看做车间，图看做车床，里面用Tensor做原料，变量做容器，feed和fetch做铲子，把数据加工成我们的结果。

**导入包**

In [1]:
import tensorflow as tf

/Users/roland/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 创建图和运行图（常量的创建与计算）

**创建常量**

In [4]:
# 创建一个常量
con1=tf.constant([[2,3]])

# 创建另一个常量
con2=tf.constant([[2],[3]])

**创建矩阵乘法节点**

In [5]:
product=tf.matmul(con1,con2)

**定义&运行会话**

In [6]:
# 定义会话
sess=tf.Session()

# 执行会话得到结果
result=sess.run(product)

# 打印结果
print(result)

# 关闭会话
sess.close()

[[13]]


## 创建变量并赋值

**创建变量**

In [8]:
# 创建一个变量，初始值为0，名字是count
num=tf.Variable(0,name='count')

**创建加法操作**

In [9]:
# 加法操作，每次加10
new_value=tf.add(num,10)

# 赋值操作，把加过的值赋值给num
op=tf.assign(num,new_value)

**开启会话，并让它自动关闭**

In [11]:
with tf.Session() as sess:
    
    # 变量一定记得初始化
    sess.run(tf.global_variables_initializer())
    
    # 打印变量初始值
    print(sess.run(num))
    
    # 循环赋值
    for i in range(0,5):
        sess.run(op)
        print(sess.run(num))

0
10
20
30
40
50


在这里，我们发现，我们不用自己去操作累加的那一步（sess.run(new_value)),只需要在session中执行最后的赋值就可以了（sess.run(new_value)）。

是因为在tensorflow的设计中，它会自动去寻找执行的节点所依赖的节点，并去执行前置的依赖节点。

唔，这个就和依赖注入有点像。

## 声明变量

之前我们在声明变量的同时赋了初始值，但有的时候，如果我们不想赋初始值呢？

这个时候，就要用占位符（placeholder）。

**创建占位符**

In [12]:
# 为变量1创建一个占位符
input1=tf.placeholder(tf.float32)

# 为变量2创建一个占位符
input2=tf.placeholder(tf.float32)

**为这两个变量写好操作**

In [13]:
# 两个变量执行加法操作
new_value=tf.add(input1,input2)

**开启会话**

In [16]:
with tf.Session() as sess:
    
    # 创建给占位符输入的内容
    feed_dict={input1:32.1,input2:23.2}
    
    # 执行节点
    result=sess.run(new_value,feed_dict=feed_dict)
    
    # 打印结果
    print(result)

55.3


# 数据预处理

接下来就要用tensorflow搭建一个简单的深度学习模型来进行手写字母的识别了。

首先导入数据处理包。

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 读入数据

**读入数据**

In [157]:
train=pd.read_csv('input/train.csv',index_col=False)
test_raw_data=pd.read_csv('input/test.csv',index_col=False)

**分割label和训练集**

In [163]:
images=train_raw_data.iloc[:,1:].values
labels_flat=train_raw_data.iloc[:,0].values.ravel()

## 对训练样本做预处理

这里简单做个处理：

    1、归一化，0~255的范围太大，我们把它缩小到0~1。

**归一化处理**

In [164]:
images=images.astype(np.float)
images=np.multiply(images,1.0 / 255.0)
images

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [165]:
image_size=images.shape[1]
image_size

784

## 对训练label做处理

简单想一下，我们这个其实是一个分类问题，对于每个数字而言，都可以看做是一个one-vs-all的问题。

因此，我们的网络最终输出的是一个10*1的向量，只有一个位置是1，其他位置是0.

因此，需要对train_y做一个one_hot编码。

下面介绍两种one-hot的方式，一种是自己写代码，一种是利用pandas的函数。

自己写代码，不用通过pandas，这样内存压力小一点；pandas写得效率高一些。

**利用自己写得代码进行one-hot**

In [166]:
labels_count=np.unique(labels_flat).shape[0]
labels_count

10

In [167]:
def dense_to_one_hot(labels_dense,num_classes):
    num_labels=labels_dense.shape[0]
    # 计算每一行所需要的偏移量
    index_offset=np.arange(num_labels)*num_classes
    labels_one_hot=np.zeros((num_labels,num_classes))
    # 把一个n*n的矩阵看做是一个1*n²的行向量，进行赋值
    labels_one_hot.flat[index_offset+labels_dense.ravel()]=1
    # 得到one-hot
    return labels_one_hot

labels=dense_to_one_hot(labels_flat,labels_count)
labels=labels.astype(np.uint8)
labels

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

**利用pandas进行one-hot**

In [169]:
# # 重新拿到数据
# train_y_one_hot=images.iloc[:,0]

# # 进行one-hot
# train_y_one_hot_real=pd.get_dummies(train_y_one_hot,prefix='Num:')

In [170]:
# # 从dataframe转化成numpy形式
# result_one_hot=train_y_one_hot_real.values
# # 输出
# result_one_hot

## 训练集划分

神经网络需要调整许多的超参数，因此我们需要把训练集再分割，分成训练集和验证集两部分。

In [171]:
VALIDATION_SIZE=2000

validation_images=train_x_nor[:VALIDATION_SIZE]
validation_labels=result_one_hot[:VALIDATION_SIZE]

train_images=train_x_nor[VALIDATION_SIZE:]
train_labels=result_one_hot[VALIDATION_SIZE:]

# train_labels.shape

## 随机梯度下降

呃，一次使用所有的数据行进行训练慢，而且内存吃不消，因此采用随机梯度下降，即批量梯度下降的方式来进行。

In [190]:
batch_size=50
n_batch=int(len(train_images)/batch_size)

# 搭建神经网络模型

我们的神经网络的超参数：

    1. 激活函数：softmax
    2. 损失函数：交叉熵
    3. 优化方式：梯度下降
    4. 输入层：784个节点，对应每个像素点。
    5. 隐层：只有一个隐层，10个节点，同时作为输出层。
    6. 输出层：就是隐层。

**训练数据的占位符**

In [191]:
image_size=train_images.shape[1]
x=tf.placeholder(tf.float32,shape=[None,image_size])
x2=tf.placeholder(tf.float32,shape=[None,image_size])
y = tf.placeholder(tf.float32, shape=[None, labels_count])

**中间参数**

In [192]:
# 这个weights表示第一隐层只有10个节点
weights=tf.Variable(tf.zeros([784,10]))

# biases,偏差，相当于y=ax+b中的b
biases=tf.Variable(tf.zeros([10]))

# result,定义第一隐层的输出
result=tf.matmul(x,weights)+biases

# prediction，激活函数，对线性计算结果添加非线性内容。
prediction=tf.nn.softmax(result)

**创建损失函数**

In [193]:
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

**用梯度下降来进行优化**

学习率为0.1，最小化loss

In [194]:
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

**初始化变量**

In [195]:
init=tf.global_variables_initializer()

**计算精度**

In [196]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

**开始训练**

In [197]:
new_value=tf.add(x,x2)
with tf.Session() as sess:
    # 初始化变量
    sess.run(init)
    
    # 循环50轮
    for epoch in range(50):
        for batch in range(n_batch):
            # 按照batch取出数据
            batch_x=train_images[batch*batch_size:(batch+1)*batch_size]
            batch_y=train_labels[batch*batch_size:(batch+1)*batch_size]
            # 开始训练
            sess.run(train_step,feed_dict={x:batch_x,y:batch_y})
        # 每一轮训练计算结果
        accuracy_n=sess.run(accuracy,feed_dict={x:validation_images,y:validation_labels})
        # 打印结果
        print("第"+str(epoch+1)+"轮，准确度为："+str(accuracy_n))

第1轮，准确度为：0.54
第2轮，准确度为：0.693
第3轮，准确度为：0.771
第4轮，准确度为：0.7835
第5轮，准确度为：0.793
第6轮，准确度为：0.7985
第7轮，准确度为：0.803
第8轮，准确度为：0.8065
第9轮，准确度为：0.8075
第10轮，准确度为：0.8075
第11轮，准确度为：0.8105
第12轮，准确度为：0.8125
第13轮，准确度为：0.814
第14轮，准确度为：0.8155
第15轮，准确度为：0.816
第16轮，准确度为：0.8175
第17轮，准确度为：0.8175
第18轮，准确度为：0.821
第19轮，准确度为：0.8335
第20轮，准确度为：0.85
第21轮，准确度为：0.8585
第22轮，准确度为：0.862
第23轮，准确度为：0.8685
第24轮，准确度为：0.8715
第25轮，准确度为：0.8745
第26轮，准确度为：0.8795
第27轮，准确度为：0.8795
第28轮，准确度为：0.881
第29轮，准确度为：0.884
第30轮，准确度为：0.886
第31轮，准确度为：0.8875
第32轮，准确度为：0.8885
第33轮，准确度为：0.89
第34轮，准确度为：0.8905
第35轮，准确度为：0.8915
第36轮，准确度为：0.891
第37轮，准确度为：0.8915
第38轮，准确度为：0.8935
第39轮，准确度为：0.894
第40轮，准确度为：0.894
第41轮，准确度为：0.8945
第42轮，准确度为：0.895
第43轮，准确度为：0.895
第44轮，准确度为：0.895
第45轮，准确度为：0.8955
第46轮，准确度为：0.8965
第47轮，准确度为：0.897
第48轮，准确度为：0.897
第49轮，准确度为：0.898
第50轮，准确度为：0.8985
